In [ ]:
#different versio of numpy required for top2vec
!pip uninstall numpy
!pip install numpy==1.19.5

In [7]:
#install the packages
# !pip install sentence-transformers
# !pip install nltk
# !pip install torch
# !pip install tensorflow
# !pip install tensorboard
!pip install top2vec
!pip install top2vec[sentence_encoders]
# !pip install top2vec[sentence_transformers]
# !pip install top2vec[indexing]

  Using cached https://files.pythonhosted.org/packages/73/ef/8967d406f3f85018ceb5efab50431e901683188f1741ceb053efcab26c87/numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [6]:
#import required packages
import random
from tqdm import tqdm
import numpy as np
import os
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize


In [2]:
# !git clone https://github.com/akashjorss/sentence_bert
# !unzip sentence_bert/sentences.zip
import pickle
with open('sentences.bin', 'rb') as f:
  sentences = pickle.load(f)

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
#load tensorboards with magics
%tensorflow_version 2.x
%load_ext tensorboard

In [9]:
documents = random.choices(sentences,k = 10000)
len(documents)

10000

In [10]:
from top2vec import Top2Vec
model = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=4)

2021-05-06 15:50:27,951 - top2vec - INFO - Pre-processing documents for training
2021-05-06 15:50:28,543 - top2vec - INFO - Downloading universal-sentence-encoder model
2021-05-06 15:50:39,585 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2021-05-06 15:50:41,656 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2021-05-06 15:51:30,006 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2021-05-06 15:51:30,556 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [ ]:
model.document_vectors.shape

In [ ]:
num_topics = model.get_num_topics()
# model.get_topic_hierarchy()
# topic_sizes, topic_nums = model.get_topic_sizes()
print("Number of topics: ", num_topics)

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["breakfast", "dinner"], num_topics=5)
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
#get document vectors and plot using umap and tensorboard
#dim reduction using umap
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=3, 
                            metric='cosine').fit_transform(model.document_vectors)

In [ ]:
umap_embeddings.shape

In [ ]:
#visualize high dim data in embedding projector
# Set up a logs directory, so Tensorboard knows where to look for files
import numpy as np
log_dir='./logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save the weights we want to analyse as a variable. 
umap_weights = tf.Variable(umap_embeddings)
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=umap_weights)
checkpoint.save(os.path.join(log_dir, "umap_embedding.ckpt"))

# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
%tensorboard --logdir ./logs/